In [8]:
import pandas as pd
import numpy as np
import os
import pickle
import math

In [74]:
cwd = os.getcwd()
cancel = pd.read_csv(cwd + '/cancel.csv')

"A" = "Carrier" <br> 
> The cause of the cancellation or delay was due to circumstances within the airline's control (e.g. maintenance or crew problems, etc.).<br>

"B" = "Weather" <br> 
> Significant meteorological conditions (actual or forecasted) that, in the judgment of the carrier, delays or prevents the operation of a flight such as tornado, blizzard or hurricane.<br>

"C" = "National Air System" <br>
> Cancellations attributable to the national aviation system that refer to a broad set of conditions, such as non-extreme weather conditions, airport operations, heavy traffic volume, and air traffic control.<br>

"D" = "Security" <br>
> Cancellations caused by evacuation of a terminal or concourse, re-boarding of aircraft because of security breach, inoperative screening equipment and/or long lines in excess of 29 minutes at screening areas.<br>

In [3]:
# cancel['Freq'] = cancel.groupby(['ORIGIN'])['OP_UNIQUE_CARRIER'].transform('count')

In [75]:
dept = cancel['CRS_DEP_TIME'].apply(lambda x: round(x/100,2))

pos = 0
for d in dept:
    dec = (math.modf(d))
    if round(dec[0], 2) >= .29:
        dept[pos] = int(dept[pos]) +1
        if dept[pos] == 24:
            dept[pos] = 0
    else:
        dept[pos] = dept[pos]
    pos +=1
   

cancel['HOUR'] = dept.astype(int)

In [80]:
D = cancel[cancel['CANCELLATION_CODE'] == 'D']

# cancel = cancel[cancel['Freq'] > 10] 
#removed any airport used less than 10 times within the year
# excpet for D bec so few security problems
# went from 110320 rows to 109983 rows
A = cancel[cancel['CANCELLATION_CODE'] == 'A']
B = cancel[cancel['CANCELLATION_CODE'] == 'B']
C = cancel[cancel['CANCELLATION_CODE'] == 'C']

## CODE: A (Carrier)

In [94]:
# A has 28085 rows
# 273 unique airports

In [5]:
#certain planes that might be more prone to canceltaion
A['TAIL_NUM'].value_counts()[:15]

N719EV    66
N738EV    60
N226JS    52
N248PS    51
N603QX    49
N244PS    48
N250PS    48
N258PS    48
N251PS    47
N617QX    47
N600QX    45
N611QX    44
N230PS    42
N720EV    42
N223JS    41
Name: TAIL_NUM, dtype: int64

In [11]:
A['OP_UNIQUE_CARRIER'].value_counts()

WN    5372
OH    4425
AA    4176
YV    2216
YX    1663
MQ    1519
F9    1476
B6    1377
EV    1195
AS     985
UA     891
OO     772
DL     565
G4     396
NK     331
9E     314
HA     256
VX     224
Name: OP_UNIQUE_CARRIER, dtype: int64

In [9]:
A['MONTH'].value_counts()

6     5959
7     3391
8     3094
3     2348
1     2318
5     2201
9     1881
4     1867
2     1726
10    1638
12     990
11     672
Name: MONTH, dtype: int64

In [15]:
A['ORIGIN'].value_counts()[:5]

CLT    1939
DFW    1406
LAX    1034
LGA     975
ORD     815
Name: ORIGIN, dtype: int64

In [16]:
A['DEST'].value_counts()[:5]

CLT    2388
DFW    1740
LAX    1068
ORD    1025
LGA    1008
Name: DEST, dtype: int64

In [31]:
A['HOUR'].value_counts()

6.0     2036
7.0     1963
8.0     1884
18.0    1756
9.0     1717
12.0    1715
15.0    1639
17.0    1631
11.0    1611
10.0    1585
13.0    1568
19.0    1529
16.0    1524
20.0    1510
14.0    1507
22.0     964
21.0     949
23.0     440
5.0      389
24.0     103
1.0       68
0.0       24
2.0       15
3.0       14
4.0       12
Name: HOUR, dtype: int64

In [123]:
# get frequency of canceled airline and airport combo
J = A.groupby(['OP_UNIQUE_CARRIER', 'ORIGIN']).size()
aline = A['OP_UNIQUE_CARRIER'].unique()
aport = A['ORIGIN'].unique()

In [6]:
a_combo = []
for l in aline:
    for a in aport:
        if a in J[l]:
            if J[l][a] > 100:
                a_combo.append([J[l][a], l, a])

In [15]:
worst_a_line_port = sorted(a_combo, reverse=True)

In [16]:
worst_a_line_port[:5] 

[[1363, 'OH', 'CLT'],
 [572, 'WN', 'LAX'],
 [498, 'AA', 'DFW'],
 [495, 'YV', 'DFW'],
 [434, 'WN', 'OAK']]

In [121]:
# get frequency of canceled airline, airport and month combo
M = A.groupby(['OP_UNIQUE_CARRIER', 'ORIGIN', 'MONTH']).size()

In [124]:
a_trio = []
for l in aline:
    for a in aport:
        if a in M[l]:
            for m in range(1,13):
                if m in M[l][a]:
                    if M[l][a][m] > 50:
                        a_trio.append([M[l][a][m], l, a, m])

In [125]:
worst_a_line_port_month = sorted(a_trio, reverse=True)

In [126]:
worst_a_line_port_month[:5] 

[[1161, 'OH', 'CLT', 6],
 [180, 'OH', 'DCA', 6],
 [147, 'AA', 'DFW', 6],
 [131, 'OH', 'PHL', 6],
 [113, 'YX', 'LGA', 3]]

In [ ]:
# giant glitch with computer sytem and PSA airlines ('OH' a subsidiary of 'AA') during June
# mainly in Charlotte, this skewed a lot of data for cancellation

## CODE: B (Weather)

In [ ]:
# 57756
# 338 airports from

In [41]:
38878/57756

0.673142184361798

In [39]:
B['MONTH'].value_counts()
# About 67% of weather cancellations happened within the first three months of the year
# showing winter is the strongest factor for these cancellations

3     13566
1     12656
2      5753
9      4609
7      3479
4      3233
5      3217
8      3159
12     2947
6      2486
10     2147
11      504
Name: MONTH, dtype: int64

In [37]:
B['OP_UNIQUE_CARRIER'].value_counts()

WN    9774
AA    8535
OO    7760
OH    5362
YX    4813
DL    3864
B6    3601
UA    3265
MQ    2204
YV    2105
9E    2009
EV    1389
NK    1289
F9     752
AS     576
G4     243
VX     201
HA      14
Name: OP_UNIQUE_CARRIER, dtype: int64

In [9]:
B['ORIGIN'].value_counts()[:10]

ORD    3295
LGA    3029
CLT    2869
DFW    2592
BOS    2393
ATL    2327
EWR    2181
DCA    2046
PHL    1857
MDW    1558
Name: ORIGIN, dtype: int64

In [81]:
B['HOUR'].value_counts()

18    4825
20    4164
19    4011
6     3918
17    3896
16    3509
15    3383
7     3365
13    3018
8     3014
14    2899
9     2833
12    2816
11    2795
10    2643
21    2632
22    2286
23     919
5      567
0      177
1       58
3       11
4       10
2        7
Name: HOUR, dtype: int64

In [12]:
B['DEST'].value_counts()[:10]

ORD    3442
CLT    2963
LGA    2929
DFW    2661
ATL    2400
BOS    2345
EWR    2211
DCA    1961
PHL    1857
JFK    1535
Name: DEST, dtype: int64

In [11]:
B['ORIGIN_CITY_NAME'].value_counts()[:10]

Chicago, IL              4853
New York, NY             4548
Charlotte, NC            2869
Dallas/Fort Worth, TX    2592
Washington, DC           2535
Boston, MA               2393
Atlanta, GA              2327
Newark, NJ               2181
Philadelphia, PA         1857
Minneapolis, MN          1312
Name: ORIGIN_CITY_NAME, dtype: int64

In [31]:
B['DEST_CITY_NAME'].value_counts()[:10]

Chicago, IL              4971
New York, NY             4464
Charlotte, NC            2963
Dallas/Fort Worth, TX    2661
Washington, DC           2457
Atlanta, GA              2400
Boston, MA               2345
Newark, NJ               2211
Philadelphia, PA         1857
Houston, TX              1360
Name: DEST_CITY_NAME, dtype: int64

In [43]:
not_cold = B[(B['ORIGIN_CITY_NAME'] != 'Chicago, IL') &(B['ORIGIN_CITY_NAME'] != 'New York, NY')\
             & (B['DEST_CITY_NAME'] !='Chicago, IL')&(B['DEST_CITY_NAME'] !='New York, NY')]

In [90]:
city = B.reindex(columns= (['ORIGIN_CITY_NAME', 'DEST_CITY_NAME']))

In [91]:
cities = city.melt()['value'].value_counts()

In [113]:
cities[:10]

Chicago, IL              9824
New York, NY             9012
Charlotte, NC            5832
Dallas/Fort Worth, TX    5253
Washington, DC           4992
Boston, MA               4738
Atlanta, GA              4727
Newark, NJ               4392
Philadelphia, PA         3714
Houston, TX              2663
Name: value, dtype: int64

In [ ]:
# 31% of cancellations involved Chicago or New York

In [92]:
used_city = cities.where(cities > 50).dropna().index.tolist()

In [89]:
Weather = B.groupby(['ORIGIN_CITY_NAME', 'DEST_CITY_NAME', 'MONTH']).size()
# B origin
borg = B['ORIGIN_CITY_NAME'].unique()
# B Destinations
bdest = B['DEST_CITY_NAME'].unique()

In [141]:
b_trio = []
for o in borg:
    if o in used_city:
        for d in bdest:
            if (d in used_city) & (d in Weather[o]):
                for m in range(1,13):
                    if m in Weather[o][d]:
                        if Weather[o][d][m] > 50:
                            b_trio.append([Weather[o][d][m], o, d, m])

In [142]:
sorted(b_trio, reverse=True)

[[186, 'Boston, MA', 'New York, NY', 3],
 [180, 'New York, NY', 'Boston, MA', 3],
 [145, 'New York, NY', 'Chicago, IL', 3],
 [133, 'Chicago, IL', 'New York, NY', 3],
 [114, 'Washington, DC', 'Boston, MA', 3],
 [114, 'Boston, MA', 'Washington, DC', 3],
 [113, 'New York, NY', 'Washington, DC', 3],
 [109, 'Washington, DC', 'New York, NY', 3],
 [82, 'New York, NY', 'Atlanta, GA', 3],
 [81, 'New York, NY', 'Chicago, IL', 1],
 [76, 'Chicago, IL', 'New York, NY', 1],
 [76, 'Atlanta, GA', 'New York, NY', 3],
 [71, 'New York, NY', 'Boston, MA', 1],
 [70, 'Charlotte, NC', 'New York, NY', 3],
 [69, 'Boston, MA', 'New York, NY', 1],
 [68, 'New York, NY', 'Miami, FL', 3],
 [67, 'Philadelphia, PA', 'Boston, MA', 3],
 [67, 'New York, NY', 'Charlotte, NC', 3],
 [67, 'Boston, MA', 'Philadelphia, PA', 3],
 [66, 'Boston, MA', 'Baltimore, MD', 3],
 [62, 'Miami, FL', 'New York, NY', 3],
 [62, 'Baltimore, MD', 'Boston, MA', 3],
 [59, 'Boston, MA', 'Chicago, IL', 3],
 [58, 'New York, NY', 'Dallas/Fort Worth,

In [ ]:
# majority frequent weather cancellations involved New York or Chicago (all cancellations if include Boston)
# all frequent destination/origin cancellations happened in Jan, Feb, or March (with majority coming in March and only one in anohter month)
# be especially weary of any BOS-NY or NY-CHI combinations during the winter

In [103]:
# get frequency of canceled origin, month, hour combo
w_hour = B.groupby(['MONTH', 'HOUR','ORIGIN']).size()

org = B['ORIGIN'].unique()

In [116]:
b_hr= []
for m in range(1,13):
    for h in range(24):
        if h in w_hour[m]:
            for o in org:
                if o in w_hour[m][h]:
                    if w_hour[m][h][o] >= 75:
                        b_hr.append([w_hour[m][h][o], o, m, h])


In [117]:
sorted(b_hr, reverse=True)

[[145, 'PHL', 3, 18],
 [131, 'BOS', 3, 6],
 [122, 'BOS', 3, 7],
 [100, 'DFW', 9, 19],
 [95, 'PHL', 3, 16],
 [92, 'BOS', 3, 8],
 [90, 'DCA', 3, 20],
 [89, 'BOS', 1, 7],
 [87, 'LGA', 3, 9],
 [86, 'LGA', 3, 15],
 [86, 'DFW', 9, 21],
 [82, 'PHL', 3, 11],
 [82, 'ORD', 5, 20],
 [82, 'LGA', 3, 10],
 [81, 'CLT', 1, 20],
 [80, 'LGA', 3, 20],
 [80, 'CLT', 1, 18],
 [80, 'ATL', 12, 20],
 [79, 'BOS', 3, 18],
 [78, 'DCA', 3, 17],
 [78, 'BOS', 3, 19],
 [77, 'PHL', 3, 13],
 [77, 'ORD', 2, 17],
 [77, 'LGA', 3, 19],
 [77, 'CLT', 9, 8],
 [76, 'DCA', 3, 15],
 [75, 'LGA', 3, 7]]

## CODE: C (National Air System)

In [ ]:
# 24338 rows
# 269 airports

In [165]:
C['ORIGIN_CITY_NAME'].value_counts()[:10]

New York, NY             3992
Chicago, IL              2216
Newark, NJ               1415
Washington, DC            997
Atlanta, GA               896
Boston, MA                743
San Francisco, CA         719
Baltimore, MD             707
Dallas/Fort Worth, TX     637
Philadelphia, PA          573
Name: ORIGIN_CITY_NAME, dtype: int64

In [174]:
C['ORIGIN'].value_counts()[:10]

LGA    2618
ORD    2049
EWR    1415
JFK    1374
ATL     896
DCA     837
BOS     743
SFO     719
BWI     707
DFW     637
Name: ORIGIN, dtype: int64

In [166]:
C['OP_UNIQUE_CARRIER'].value_counts()

MQ    5581
9E    3661
YX    2823
WN    2792
EV    2464
OO    1674
B6    1311
DL    1082
AA     721
OH     704
YV     549
AS     403
NK     217
UA     217
VX     106
G4      33
Name: OP_UNIQUE_CARRIER, dtype: int64

In [178]:
C['MONTH'].value_counts()
# about 50% of the C cancellations were in the summer

8     6094
7     4204
6     2238
1     2189
5     1734
9     1606
2     1486
12    1380
3     1358
4     1144
10     697
11     208
Name: MONTH, dtype: int64

In [127]:
NAS = C.groupby(['OP_UNIQUE_CARRIER','ORIGIN','MONTH']).size()
# B origin
ccar = C['OP_UNIQUE_CARRIER'].unique()
# B Destinations
cport = C['ORIGIN'].unique()

In [149]:
port = C['ORIGIN'].value_counts()
port = port[port > 10].dropna()

In [151]:
c_trio = []
for l in ccar:
    for a in cport:
        if (a in port) & (a in NAS[l]):
            for m in range(1,13):
                if m in NAS[l][a]:
                    if NAS[l][a][m] > 50:
                        c_trio.append([NAS[l][a][m], l, a, m])

In [153]:
NAS_port_month =sorted(c_trio, reverse=True)

In [154]:
NAS_port_month[:10]

[[446, 'DL', 'ATL', 12],
 [310, 'MQ', 'ORD', 6],
 [257, '9E', 'LGA', 8],
 [251, 'YX', 'LGA', 8],
 [233, 'WN', 'BWI', 7],
 [218, '9E', 'JFK', 8],
 [213, 'MQ', 'ORD', 2],
 [202, 'EV', 'EWR', 8],
 [192, 'MQ', 'ORD', 8],
 [192, '9E', 'JFK', 1]]

In [133]:
dest_port = C['DEST'].value_counts()
arr = dest_port[dest_port > 50].dropna()

In [134]:
NAS_dest = C.groupby(['OP_UNIQUE_CARRIER','DEST','MONTH']).size()
cdest = C['DEST'].unique()

In [144]:
c_dtrio = []
for l in ccar:
    for a in cdest:
        if (a in arr) & (a in NAS_dest[l]):
            for m in range(1,13):
                if m in NAS_dest[l][a]:
                    if NAS_dest[l][a][m] > 50:
                        c_dtrio.append([NAS_dest[l][a][m], l, a, m])

In [136]:
sorted(c_dtrio, reverse=True)[:10]

[[445, 'DL', 'ATL', 12],
 [332, 'MQ', 'ORD', 6],
 [252, '9E', 'LGA', 8],
 [243, 'YX', 'LGA', 8],
 [232, 'WN', 'BWI', 7],
 [218, 'MQ', 'ORD', 2],
 [216, '9E', 'JFK', 8],
 [208, 'EV', 'EWR', 8],
 [201, '9E', 'JFK', 1],
 [193, 'MQ', 'ORD', 8]]

In [ ]:
# There was a power outage at Hartsfield-Jackson Atlanta International Airport on Dec 17
# earning 'ATL' the the top spots by most cancled 

In [118]:
# figured the later in the day, the more cancled
C['HOUR'].value_counts()

18    2588
17    2438
19    2266
16    2003
15    1868
20    1855
14    1346
21    1270
13    1227
12    1192
22    1018
6      956
11     901
9      843
10     798
8      730
7      722
23     209
5       65
0       33
4        3
3        3
1        3
2        1
Name: HOUR, dtype: int64

In [155]:
N_hr = C.groupby(['OP_UNIQUE_CARRIER','ORIGIN','HOUR']).size()

In [157]:
c_time = []
for l in ccar:
    for o in cport:
        if (o in port) & (o in N_hr[l]):
            for h in range(24):
                if h in N_hr[l][o]:
                    if N_hr[l][o][h] > 50:
                        c_time.append([N_hr[l][o][h], l, o, h])

In [160]:
sorted(c_time, reverse=True)[:10]

[[212, 'MQ', 'ORD', 17],
 [159, 'MQ', 'ORD', 19],
 [152, 'MQ', 'ORD', 15],
 [125, 'EV', 'EWR', 22],
 [116, 'MQ', 'ORD', 22],
 [111, 'MQ', 'JFK', 16],
 [105, '9E', 'JFK', 22],
 [100, '9E', 'LGA', 21],
 [98, 'WN', 'BWI', 20],
 [97, '9E', 'JFK', 19]]

## CODE: D (Security)

In [ ]:
# 73 rows

In [240]:
D['ORIGIN'].value_counts()

PHX    16
MCO    15
CLT     4
DFW     3
BLV     3
ORD     3
LAX     2
SFO     2
PVD     1
SBA     1
LGA     1
SJC     1
PIA     1
JAX     1
MIA     1
DCA     1
EWR     1
BOS     1
IAH     1
JFK     1
IND     1
TUS     1
SMF     1
AUS     1
VPS     1
MSP     1
DEN     1
TPA     1
SAN     1
SBP     1
PHL     1
MKE     1
HPN     1
Name: ORIGIN, dtype: int64

In [241]:
D['DEST'].value_counts()

MCO    14
PHX    12
CLT     5
ORD     3
LAX     3
DEN     2
DFW     2
AUS     2
TUS     2
TPA     2
BLV     2
ORF     2
PHL     2
SBA     1
JAX     1
PIA     1
EWR     1
FLL     1
SJC     1
LGA     1
SFO     1
PVD     1
SBP     1
JFK     1
IND     1
IAH     1
SMF     1
MIA     1
VPS     1
MSP     1
SAN     1
MKE     1
HPN     1
Name: DEST, dtype: int64

In [243]:
D['MONTH'].value_counts()

9     35
11    29
4      5
2      2
3      1
1      1
Name: MONTH, dtype: int64

In [269]:
PHX= D[(D['ORIGIN'] == 'PHX') | (D['DEST'] == 'PHX')]
MCO= D[(D['ORIGIN'] == 'MCO') | (D['DEST'] == 'MCO')]

In [ ]:
# MCO airport had security alert in Nov of 2017 bec of battery that exploded in someones bag(thought was a gunshot)
# PHX airport had security alert in Sept of 2018 bec if suspicious abandoned car

In [270]:
len(PHX)

28

In [271]:
len(MCO)

29

In [278]:
other= D[(D['ORIGIN'] != 'PHX') & (D['DEST'] != 'PHX')& \
    (D['ORIGIN'] != 'MCO') & (D['DEST'] != 'MCO')]

In [281]:
other

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,CRS_DEP_TIME,CRS_ARR_TIME,CANCELLATION_CODE,TOTAL_ADD_GTIME
12572,2018,1,29,1,2018-01-29,NK,N512NK,799,BOS,"Boston, MA",TPA,"Tampa, FL",2035,1,D,NaN
26429,2018,2,17,6,2018-02-17,OH,N253PS,5431,CLT,"Charlotte, NC",PIA,"Peoria, IL",1110,1214,D,NaN
26692,2018,2,17,6,2018-02-17,OH,N245PS,5549,PIA,"Peoria, IL",CLT,"Charlotte, NC",730,1035,D,NaN
44481,2018,3,20,2,2018-03-20,YX,N410YX,4438,DCA,"Washington, DC",PHL,"Philadelphia, PA",1550,1659,D,NaN
51380,2018,4,1,7,2018-04-01,G4,NaN,618,BLV,"Belleville, IL",JAX,"Jacksonville, FL",1431,1725,D,NaN
51381,2018,4,1,7,2018-04-01,G4,NaN,1352,VPS,"Valparaiso, FL",BLV,"Belleville, IL",1444,1632,D,NaN
51382,2018,4,1,7,2018-04-01,G4,NaN,619,JAX,"Jacksonville, FL",BLV,"Belleville, IL",1805,1903,D,NaN
51383,2018,4,1,7,2018-04-01,G4,NaN,1353,BLV,"Belleville, IL",VPS,"Valparaiso, FL",1717,1900,D,NaN
51384,2018,4,1,7,2018-04-01,G4,NaN,1767,BLV,"Belleville, IL",FLL,"Fort Lauderdale, FL",920,1305,D,NaN
99732,2018,9,16,7,2018-09-16,AA,N659AW,649,DFW,"Dallas/Fort Worth, TX",TUS,"Tucson, AZ",2225,2244,D,NaN
